# Predicció de càncer de pell utilitzant Machine Learning

Llibreries que utilitzarem

In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets.folder import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

Importar les dades que utilitzarem. Al ser imatges fem servir el mètode ImageFolder de Torchvision

In [ ]:
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = 'data'